In [6]:
import pandas as pd
from datetime import datetime
import os

# Path to the directory containing chunk files
# folder_path = r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks"

# # List all CSV files in the folder
# csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# # Read and concatenate all CSVs into one DataFrame
# transactions_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Source files\transactions.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")
customer_df


,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...
594638,'C1753498738','M1823072687',20.53,'es_transportation',0
594639,'C650108285','M1823072687',50.73,'es_transportation',0
594640,'C123623130','M349281107',22.44,'es_fashion',0
594641,'C1499363341','M1823072687',14.46,'es_transportation',0


In [7]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','Weight']
)
merged_df

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud_x,Source,Target,Weight,typeTrans,fraud_y
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,'C1093826151','M348934600',4.55,'es_transportation',0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,'C352968107','M348934600',39.68,'es_transportation',0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,'C2054744914','M1823072687',26.89,'es_transportation',0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,'C1760612790','M348934600',17.25,'es_transportation',0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602744,179,'C1753498738','3','F','28007','M1823072687','28007','es_transportation',20.53,0,'C1753498738','M1823072687',20.53,'es_transportation',0
602745,179,'C650108285','4','F','28007','M1823072687','28007','es_transportation',50.73,0,'C650108285','M1823072687',50.73,'es_transportation',0
602746,179,'C123623130','2','F','28007','M349281107','28007','es_fashion',22.44,0,'C123623130','M349281107',22.44,'es_fashion',0
602747,179,'C1499363341','5','M','28007','M1823072687','28007','es_transportation',14.46,0,'C1499363341','M1823072687',14.46,'es_transportation',0


In [8]:
from datetime import datetime

def detect_pattern2(transactions_df):
    detections = []

    # Step 1: Group by customer and merchant
    group = transactions_df.groupby(['customer', 'merchant']).agg(
        txn_count=('amount', 'count'),
        avg_amount=('amount', 'mean')
    ).reset_index()
    
    # Step 2: Filter on criteria
    filtered = group[( group['txn_count'] >= 80)&(group['avg_amount'] < 23)]

    # Step 3: Construct detection rows
    detection_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    y_start_time = detection_time  # assuming same for now

    for _, row in filtered.iterrows():
        print(f"txn_count: {row['txn_count']}, avg_amount: {row['avg_amount']}")

        detection = {
            "YStartTime": y_start_time,
            "detectionTime": detection_time,
            "patternId": "PatId2",
            "ActionType": "CHILD",
            "customerName": row['customer'],
            "MerchantId": row['merchant'],
            "txn_count": row['txn_count'],
        }
        detections.append(detection)

    return pd.DataFrame(detections)


In [9]:
detected_pat2_df = detect_pattern2(transactions_df)
print(detected_pat2_df.head())


txn_count: 87, avg_amount: 22.711494252873564
txn_count: 123, avg_amount: 22.981463414634145
txn_count: 80, avg_amount: 22.187625
txn_count: 105, avg_amount: 22.47752380952381
txn_count: 88, avg_amount: 22.605113636363637
txn_count: 119, avg_amount: 22.62705882352941
txn_count: 136, avg_amount: 22.57316176470588
txn_count: 101, avg_amount: 22.06049504950495
txn_count: 86, avg_amount: 22.590232558139533
txn_count: 92, avg_amount: 20.968586956521737
txn_count: 88, avg_amount: 22.753295454545455
txn_count: 101, avg_amount: 22.729900990099008
txn_count: 84, avg_amount: 22.741309523809523
txn_count: 96, avg_amount: 22.90822916666667
txn_count: 111, avg_amount: 22.954684684684683
txn_count: 91, avg_amount: 21.954505494505494
txn_count: 93, avg_amount: 21.346129032258066
txn_count: 103, avg_amount: 22.75106796116505
txn_count: 124, avg_amount: 22.874919354838713
txn_count: 85, avg_amount: 22.507882352941177
txn_count: 91, avg_amount: 20.77824175824176
txn_count: 117, avg_amount: 22.6082051282

In [10]:
detected_pat2_df

,YStartTime,detectionTime,patternId,ActionType,customerName,MerchantId,txn_count
0,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1050458046','M1823072687',87
1,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C107125195','M1823072687',123
2,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1098443227','M1823072687',80
3,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1147125608','M1823072687',105
4,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1186894186','M348934600',88
5,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1200227736','M1823072687',119
6,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1214229415','M1823072687',136
7,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1240209240','M1823072687',101
8,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1273110804','M1823072687',86
9,2025-07-03 02:50:37,2025-07-03 02:50:37,PatId2,CHILD,'C1275741147','M1823072687',92
